## 1. Downloading rainfall in India

### Downloading rainfall data

We will first download rainfall data for Bihar for 10 years (during the monsoon season of June to September) from the Indian Meteorological Department. 


In [20]:
#!pip install imdlib
#!pip install netCDF4
#!pip install scikit-learn
#!pip install lightgbm
!pip install openmeteo_requests retry_requests


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 KB 4.5 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.2/683.2 KB 10.1 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.8 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 KB 17.0 MB/s eta 0:00:00


In [5]:
import imdlib as imd

# Downloading 8 years of rainfall data for India
start_yr = 2014
end_yr = 2023
variable = 'rain' # other options are ('tmin'/ 'tmax')
file_dir = 'dataFolder'
data = imd.get_data(variable, start_yr, end_yr, fn_format='yearwise', file_dir=file_dir)


Downloading: rain for year 2014
Downloading: rain for year 2015
Downloading: rain for year 2016
Downloading: rain for year 2017
Downloading: rain for year 2018
Downloading: rain for year 2019
Downloading: rain for year 2020
Downloading: rain for year 2021
Downloading: rain for year 2022
Downloading: rain for year 2023
Download Successful !!!


In [18]:
import xarray as xr

ds = data.get_xarray()
type(ds)
lat_min, lat_max = 24.5, 27.5
lon_min, lon_max = 83.0, 88.0
spatial_subset = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))

months = [6, 7, 8, 9]
time_subset = spatial_subset.where(spatial_subset['time.month'].isin(months), drop=True)

time_subset.to_netcdf('Bihar_June-Sept.nc')
time_subset

<xarray.Dataset> Size: 3MB
Dimensions:  (time: 1220, lat: 13, lon: 21)
Coordinates:
  * lat      (lat) float64 104B 24.5 24.75 25.0 25.25 ... 26.75 27.0 27.25 27.5
  * lon      (lon) float64 168B 83.0 83.25 83.5 83.75 ... 87.25 87.5 87.75 88.0
  * time     (time) datetime64[ns] 10kB 2014-06-01 2014-06-02 ... 2023-09-30
Data variables:
    rain     (time, lat, lon) float64 3MB 0.0 0.0 0.0 ... -999.0 -999.0 0.4226
Attributes:
    Conventions:  CF-1.7
    title:        IMD gridded data
    source:       https://imdpune.gov.in/
    history:      2025-12-08 18:48:45.910574 Python
    references:   
    comment:      
    crs:          epsg:4326

In [9]:
import xarray as xr
ds = xr.open_dataset('bihar_monsoon_2014_2023.nc')
# See dataset information
print(ds)

<xarray.Dataset>
Dimensions:  (time: 1220, lat: 13, lon: 21)
Coordinates:
  * time     (time) datetime64[ns] 2014-06-01 2014-06-02 ... 2023-09-30
  * lon      (lon) float64 83.0 83.25 83.5 83.75 84.0 ... 87.25 87.5 87.75 88.0
  * lat      (lat) float64 24.5 24.75 25.0 25.25 25.5 ... 26.75 27.0 27.25 27.5
Data variables:
    rain     (time, lat, lon) float64 ...
Attributes:
    Conventions:  CF-1.7
    title:        IMD gridded data
    source:       https://imdpune.gov.in/
    history:      2025-12-08 18:48:21.455196 Python
    references:   
    comment:      
    crs:          epsg:4326


In [22]:
"""
Create spatial average rainfall from Bihar monsoon netCDF data.

This script reads the raw netCDF file containing gridded rainfall data
and computes the daily spatial average across all grid points in Bihar.

The netCDF has dimensions (time, latitude, longitude) and we compute:
    spatial_avg[t] = mean(rainfall[t, :, :])

Output: CSV with columns [date, rainfall_mm]
"""

from pathlib import Path
import xarray as xr
import pandas as pd
import numpy as np

# Paths
ROOT = Path.cwd().parent 
RAW_DIR = ROOT / "data" / "raw"
PROCESSED_DIR = ROOT / "data" / "processed"
NC_FILE = RAW_DIR / "bihar_monsoon_2014_2023.nc"
OUTPUT_FILE = PROCESSED_DIR / "bihar_spatial_avg.csv"


def create_spatial_average(nc_path: Path) -> pd.DataFrame:
    """
    Create spatial average rainfall from netCDF file.

    Args:
        nc_path: Path to netCDF file with gridded rainfall

    Returns:
        DataFrame with columns [date, rainfall_mm]
    """
    print(f"Loading: {nc_path}")
    ds = xr.open_dataset(nc_path)

    # Print dataset info
    print("\nDataset structure:")
    print(f"  Variables: {list(ds.data_vars)}")
    print(f"  Dimensions: {dict(ds.sizes)}")
    print(f"  Coordinates: {list(ds.coords)}")

    # Find the rainfall variable (common names: precipitation, precip, rainfall, rain, rf, pr)
    rain_var = None
    possible_names = ["precipitation", "precip", "rainfall", "rain", "rf", "pr", "tp"]
    for name in possible_names:
        if name in ds.data_vars:
            rain_var = name
            break

    if rain_var is None:
        # Take the first data variable if no standard name found
        rain_var = list(ds.data_vars)[0]
        print(f"\nWarning: Using first variable '{rain_var}' as rainfall")
    else:
        print(f"\nUsing variable: '{rain_var}'")

    rainfall = ds[rain_var]
    print(f"  Shape: {rainfall.shape}")
    print(f"  Units: {rainfall.attrs.get('units', 'unknown')}")

    # Identify time dimension
    time_dim = None
    for dim in rainfall.dims:
        if dim.lower() in ["time", "t", "date"]:
            time_dim = dim
            break

    if time_dim is None:
        time_dim = rainfall.dims[0]  # Assume first dimension is time
        print(f"  Assuming '{time_dim}' is time dimension")

    # Identify spatial dimensions (everything except time)
    spatial_dims = [d for d in rainfall.dims if d != time_dim]
    print(f"  Spatial dims: {spatial_dims}")

    # Check for fill values (common: -999, -9999, etc.)
    # These are used to mark grid points outside the region of interest
    min_val = float(rainfall.min())
    if min_val < -900:  # Likely has fill value
        fill_value = min_val
        print(f"  Detected fill value: {fill_value}")
        # Mask fill values as NaN
        rainfall = rainfall.where(rainfall > -900)

    # Compute spatial mean for each time step (skipna=True by default)
    print("\nComputing spatial average...")
    spatial_avg = rainfall.mean(dim=spatial_dims, skipna=True)

    # Get time coordinate
    time_coord = ds[time_dim]

    # Convert to pandas
    df = pd.DataFrame({
        "date": pd.to_datetime(time_coord.values),
        "rainfall_mm": spatial_avg.values
    })

    # Handle any NaN values
    nan_count = df["rainfall_mm"].isna().sum()
    if nan_count > 0:
        print(f"  Warning: {nan_count} NaN values found, filling with 0")
        df["rainfall_mm"] = df["rainfall_mm"].fillna(0)

    # Ensure non-negative rainfall
    df["rainfall_mm"] = df["rainfall_mm"].clip(lower=0)

    ds.close()

    return df


def main():
    # Ensure output directory exists
    PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

    if not NC_FILE.exists():
        print(f"Error: NetCDF file not found: {NC_FILE}")
        return

    # Create spatial average
    df = create_spatial_average(NC_FILE)

    # Print summary
    print("\n" + "=" * 50)
    print("SPATIAL AVERAGE SUMMARY")
    print("=" * 50)
    print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
    print(f"Total days: {len(df)}")
    print(f"Mean rainfall: {df['rainfall_mm'].mean():.2f} mm/day")
    print(f"Max rainfall: {df['rainfall_mm'].max():.2f} mm/day")
    print(f"Total rainfall: {df['rainfall_mm'].sum():.2f} mm")
    print(f"Days with rain (>0.1mm): {(df['rainfall_mm'] > 0.1).sum()}")

    # Save
    df.to_csv(OUTPUT_FILE, index=False)
    print(f"\nSaved to: {OUTPUT_FILE}")

    # Preview
    print("\nFirst 10 rows:")
    print(df.head(10).to_string(index=False))


if __name__ == "__main__":
    main()


Loading: /home/chaiti/rainfall-prediction/rainfall-prediction/data/raw/bihar_monsoon_2014_2023.nc

Dataset structure:
  Variables: ['rain']
  Dimensions: {'time': 1220, 'lat': 13, 'lon': 21}
  Coordinates: ['time', 'lon', 'lat']

Using variable: 'rain'
  Shape: (1220, 13, 21)
  Units: mm/day
  Spatial dims: ['lat', 'lon']
  Detected fill value: -999.0

Computing spatial average...

SPATIAL AVERAGE SUMMARY
Date range: 2014-06-01 to 2023-09-30
Total days: 1220
Mean rainfall: 7.77 mm/day
Max rainfall: 72.33 mm/day
Total rainfall: 9481.04 mm
Days with rain (>0.1mm): 1184

Saved to: /home/chaiti/rainfall-prediction/rainfall-prediction/data/processed/bihar_spatial_avg.csv

First 10 rows:
      date  rainfall_mm
2014-06-01     0.173380
2014-06-02     0.067883
2014-06-03     0.473584
2014-06-04     0.553268
2014-06-05     3.200293
2014-06-06     1.044967
2014-06-07     4.491532
2014-06-08     2.362717
2014-06-09     2.537074
2014-06-10     9.617831


## 2. Using a hugging face model to look at rainfall predictions

In [13]:
import sys
import sys
import sklearn
from pathlib import Path

sys.path.insert(0, str(ROOT / "src"))

import pandas as pd
from rainfallprediction.model import load_model, predict
from rainfallprediction.features import prepare_features
from rainfallprediction.evaluate import compute_metrics, compute_metrics_by_month

# Paths
DATA_DIR = ROOT / "data"
INPUT_FILE = DATA_DIR / "processed" / "bihar_spatial_avg.csv"
OUTPUT_DIR = ROOT / "outputs"
OUTPUT_FILE = OUTPUT_DIR / "bihar_predictions.csv"


def main():
    # Ensure output directory exists
    OUTPUT_DIR.mkdir(exist_ok=True)

    # Load model
    model = load_model()

    # Load Bihar data
    df = pd.read_csv(INPUT_FILE, parse_dates=["date"])
    print(f"Bihar rainfall data: {len(df)} days")
    print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
    print()

    # Prepare features
    df_pred = prepare_features(df, rainfall_col="rainfall_mm", date_col="date")

    # Predict
    df_pred["predicted_rainfall"] = predict(model, df_pred)

    # Results
    print("=" * 60)
    print("PREDICTION RESULTS")
    print("=" * 60)
    print(f"Predictions made for: {len(df_pred)} days")
    print()

    # Sample predictions
    print("Sample predictions (first 20 days):")
    print(df_pred[["date", "rainfall_mm", "predicted_rainfall"]].head(20).to_string(index=False))
    print()

    # Metrics
    metrics = compute_metrics(df_pred["rainfall_mm"], df_pred["predicted_rainfall"])

    print("=" * 60)
    print("MODEL PERFORMANCE ON BIHAR DATA")
    print("=" * 60)
    print(f"RMSE: {metrics['rmse']:.2f} mm")
    print(f"MAE:  {metrics['mae']:.2f} mm")
    print(f"R²:   {metrics['r2']:.3f}")
    print()

    # By month
    print("Performance by Month:")
    month_names = {6: "June", 7: "July", 8: "August", 9: "September"}
    monthly = compute_metrics_by_month(df_pred, "rainfall_mm", "predicted_rainfall")
    for _, row in monthly.iterrows():
        name = month_names.get(row["month"], str(row["month"]))
        print(f"  {name:>10}: RMSE={row['rmse']:.2f} mm, MAE={row['mae']:.2f} mm")
    print()

    # Save predictions
    df_pred[["date", "rainfall_mm", "predicted_rainfall"]].to_csv(OUTPUT_FILE, index=False)
    print(f"Predictions saved to: {OUTPUT_FILE}")

    # Distribution stats
    print()
    print("=" * 60)
    print("RAINFALL DISTRIBUTION")
    print("=" * 60)
    actual = df_pred["rainfall_mm"]
    predicted = df_pred["predicted_rainfall"]
    print(f"Actual:    mean={actual.mean():.2f}, std={actual.std():.2f}, max={actual.max():.2f} mm")
    print(f"Predicted: mean={predicted.mean():.2f}, std={predicted.std():.2f}, max={predicted.max():.2f} mm")


if __name__ == "__main__":
    main()


Bihar rainfall data: 1220 days
Date range: 2014-06-01 to 2023-09-30

PREDICTION RESULTS
Predictions made for: 1213 days

Sample predictions (first 20 days):
      date  rainfall_mm  predicted_rainfall
2014-06-08     2.362717            4.433908
2014-06-09     2.537074            4.943960
2014-06-10     9.617831            4.639105
2014-06-11     2.244312            7.329529
2014-06-12     1.470683            6.404959
2014-06-13     0.658616            4.349388
2014-06-14     0.243957            5.200445
2014-06-15     0.146356            5.448652
2014-06-16     1.352018            3.778902
2014-06-17     6.002461            5.717600
2014-06-18     3.693360            6.458920
2014-06-19     8.277402            7.024574
2014-06-20     6.910142            8.444232
2014-06-21    19.565249            8.099051
2014-06-22    13.647746            7.799876
2014-06-23     8.772097           10.207117
2014-06-24     5.742474            9.354973
2014-06-25     3.059098            8.369146
2014-06

In [16]:
""" Plotting data """

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

OUTPUT_DIR = ROOT / "outputs"
PREDICTIONS_FILE = OUTPUT_DIR / "bihar_predictions.csv"


def main():
    # Load predictions
    df = pd.read_csv(PREDICTIONS_FILE, parse_dates=["date"])

    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle("Bihar Monsoon Rainfall: Model Predictions vs Actual (2014-2023)",
                 fontsize=14, fontweight="bold")

    # 1. Time series for 2023 (most recent year)
    ax1 = axes[0, 0]
    df_2023 = df[df["date"].dt.year == 2023]
    ax1.plot(df_2023["date"], df_2023["rainfall_mm"], label="Actual", alpha=0.8, linewidth=1.5)
    ax1.plot(df_2023["date"], df_2023["predicted_rainfall"], label="Predicted", alpha=0.8, linewidth=1.5)
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Rainfall (mm)")
    ax1.set_title("2023 Monsoon Season")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. Scatter plot: Actual vs Predicted
    ax2 = axes[0, 1]
    ax2.scatter(df["rainfall_mm"], df["predicted_rainfall"], alpha=0.3, s=10)
    max_val = max(df["rainfall_mm"].max(), df["predicted_rainfall"].max())
    ax2.plot([0, max_val], [0, max_val], "r--", label="Perfect prediction")
    ax2.set_xlabel("Actual Rainfall (mm)")
    ax2.set_ylabel("Predicted Rainfall (mm)")
    ax2.set_title("Actual vs Predicted")
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # 3. Monthly comparison
    ax3 = axes[1, 0]
    df["month"] = df["date"].dt.month
    monthly = df.groupby("month").agg({"rainfall_mm": "mean", "predicted_rainfall": "mean"}).reset_index()
    x = np.arange(len(monthly))
    width = 0.35
    ax3.bar(x - width/2, monthly["rainfall_mm"], width, label="Actual", alpha=0.8)
    ax3.bar(x + width/2, monthly["predicted_rainfall"], width, label="Predicted", alpha=0.8)
    ax3.set_xlabel("Month")
    ax3.set_ylabel("Mean Rainfall (mm)")
    ax3.set_title("Average Rainfall by Month")
    ax3.set_xticks(x)
    ax3.set_xticklabels(["June", "July", "August", "September"])
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis="y")

    # 4. Error distribution
    ax4 = axes[1, 1]
    errors = df["rainfall_mm"] - df["predicted_rainfall"]
    ax4.hist(errors, bins=50, alpha=0.7, edgecolor="black")
    ax4.axvline(x=0, color="red", linestyle="--", label="Zero error")
    ax4.axvline(x=errors.mean(), color="green", linestyle="--", label=f"Mean error: {errors.mean():.2f}")
    ax4.set_xlabel("Prediction Error (mm)")
    ax4.set_ylabel("Frequency")
    ax4.set_title("Error Distribution")
    ax4.legend()
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    output_path = OUTPUT_DIR / "bihar_prediction_comparison.png"
    plt.savefig(output_path, dpi=150, bbox_inches="tight")
    plt.close()
    print(f"Plot saved to: {output_path}")

    # Yearly comparison
    fig2, ax = plt.subplots(figsize=(12, 6))
    df["year"] = df["date"].dt.year
    yearly = df.groupby("year").agg({
        "rainfall_mm": ["mean", "sum"],
        "predicted_rainfall": ["mean", "sum"]
    }).reset_index()
    yearly.columns = ["year", "actual_mean", "actual_sum", "pred_mean", "pred_sum"]

    x = np.arange(len(yearly))
    width = 0.35
    ax.bar(x - width/2, yearly["actual_mean"], width, label="Actual", alpha=0.8)
    ax.bar(x + width/2, yearly["pred_mean"], width, label="Predicted", alpha=0.8)
    ax.set_xlabel("Year")
    ax.set_ylabel("Mean Daily Rainfall (mm)")
    ax.set_title("Bihar Monsoon: Yearly Average Daily Rainfall")
    ax.set_xticks(x)
    ax.set_xticklabels(yearly["year"])
    ax.legend()
    ax.grid(True, alpha=0.3, axis="y")

    plt.tight_layout()
    output_path = OUTPUT_DIR / "bihar_yearly_comparison.png"
    plt.savefig(output_path, dpi=150, bbox_inches="tight")
    plt.close()
    print(f"Plot saved to: {output_path}")


if __name__ == "__main__":
    main()

Plot saved to: /home/chaiti/rainfall-prediction/rainfall-prediction/outputs/bihar_prediction_comparison.png
Plot saved to: /home/chaiti/rainfall-prediction/rainfall-prediction/outputs/bihar_yearly_comparison.png


In [18]:
"""Extract and plot June 15-25, 2023 predictions."""
import pandas as pd
import matplotlib.pyplot as plt
OUTPUT_DIR = ROOT / "outputs"

# Load predictions
df = pd.read_csv(OUTPUT_DIR / "bihar_predictions.csv", parse_dates=["date"])

# Filter June 15-25, 2023
mask = (df["date"] >= "2023-06-15") & (df["date"] <= "2023-06-25")
df_june = df[mask].copy()

print("June 15-25, 2023 Predictions:")
print(df_june.to_string(index=False))
print()

# Save to file
output_file = OUTPUT_DIR / "june_2023_15_25.csv"
df_june.to_csv(output_file, index=False)
print(f"Saved to: {output_file}")

# Plot
fig, ax = plt.subplots(figsize=(10, 5))

x = df_june["date"]
ax.plot(x, df_june["rainfall_mm"], "o-", label="Actual", linewidth=2, markersize=8)
ax.plot(x, df_june["predicted_rainfall"], "s--", label="Predicted", linewidth=2, markersize=8)

ax.set_xlabel("Date")
ax.set_ylabel("Rainfall (mm)")
ax.set_title("Bihar Rainfall: June 15-25, 2023")
ax.legend()
ax.grid(True, alpha=0.3)

# Rotate x-axis labels
plt.xticks(rotation=45)
plt.tight_layout()

plot_file = OUTPUT_DIR / "june_2023_15_25.png"
plt.savefig(plot_file, dpi=150, bbox_inches="tight")
plt.close()
print(f"Plot saved to: {plot_file}")


June 15-25, 2023 Predictions:
      date  rainfall_mm  predicted_rainfall
2023-06-15     1.245295            6.207970
2023-06-16     1.038100            5.333638
2023-06-17     0.534527            5.421577
2023-06-18     1.918757            3.584134
2023-06-19     5.118173            6.470734
2023-06-20     2.312770            5.949299
2023-06-21     3.694649            6.515938
2023-06-22     3.028354            5.999797
2023-06-23     8.423565            5.015070
2023-06-24     0.870222            7.238396
2023-06-25     1.565293            3.934512

Saved to: /home/chaiti/rainfall-prediction/rainfall-prediction/outputs/june_2023_15_25.csv
Plot saved to: /home/chaiti/rainfall-prediction/rainfall-prediction/outputs/june_2023_15_25.png
